In [2]:
import os
import json
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import object_detector

In [3]:
!wget https://storage.googleapis.com/mediapipe-tasks/object_detector/android_figurine.zip
!unzip android_figurine.zip
train_dataset_path = "android_figurine/train"
validation_dataset_path = "android_figurine/validation"


--2024-09-06 17:31:55--  https://storage.googleapis.com/mediapipe-tasks/object_detector/android_figurine.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 216.58.214.27, 142.250.179.219, 142.251.36.27, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|216.58.214.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28607322 (27M) [application/zip]
Saving to: ‘android_figurine.zip’

android_figurine.zi 100%[===================>]  27,28M  5,26MB/s    in 6,8s    

2024-09-06 17:32:03 (4,03 MB/s) - ‘android_figurine.zip’ saved [28607322/28607322]

Archive:  android_figurine.zip
   creating: android_figurine/
   creating: android_figurine/validation/
  inflating: android_figurine/validation/labels.json  
   creating: android_figurine/validation/images/
  inflating: android_figurine/validation/images/IMG_0502.jpg  
  inflating: android_figurine/validation/images/IMG_0497.jpg  
  inflating: android_figurine/validation/images/IMG_0501.jpg  

In [4]:
with open(os.path.join(train_dataset_path, "labels.json"), "r") as f:
  labels_json = json.load(f)
for category_item in labels_json["categories"]:
  print(f"{category_item['id']}: {category_item['name']}")


0: background
1: android
2: pig_android


In [5]:
train_data = object_detector.Dataset.from_coco_folder(train_dataset_path, cache_dir="/tmp/od_data/train")
validation_data = object_detector.Dataset.from_coco_folder(validation_dataset_path, cache_dir="/tmp/od_data/validation")
print("train_data size: ", train_data.size)
print("validation_data size: ", validation_data.size)


INFO:tensorflow:On image 0


2024-09-06 17:46:20.708195: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-06 17:46:20.996679: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


INFO:tensorflow:On image 0
train_data size:  62
validation_data size:  10


In [6]:
spec = object_detector.SupportedModels.MOBILENET_MULTI_AVG
hparams = object_detector.HParams(export_dir='exported_model')
options = object_detector.ObjectDetectorOptions(
    supported_model=spec,
    hparams=hparams
)

In [7]:
model = object_detector.ObjectDetector.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options)


/home/jesse/.local/lib/python3.8/site-packages/keras/src/engine/functional.py:639: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


Model: "retina_net_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobile_net (MobileNet)      {'2': (None, 64, 64, 32   3704416   
                             ),                                  
                              '3': (None, 32, 32, 64             
                             ),                                  
                              '4': (None, 16, 16, 16             
                             0),                                 
                              '5': (None, 8, 8, 192)             
                             , '6': (None, 1, 1, 128             
                             0)}                                 
                                                                 
 fpn (FPN)                   {'5': (None, 8, 8, 128)   144928    
                             , '4': (None, 16, 16, 1             
                             28),                 

INFO:tensorflow:Training the models...


Epoch 1/30


/home/jesse/.local/lib/python3.8/site-packages/keras/src/backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


7/7 [==============================] - 42s 2s/step - total_loss: 5.9634 - cls_loss: 5.7791 - box_loss: 0.0024 - model_loss: 5.9005 - val_total_loss: 1.3662 - val_cls_loss: 1.2293 - val_box_loss: 0.0015 - val_model_loss: 1.3034
Epoch 2/30
7/7 [==============================] - 9s 1s/step - total_loss: 1.3948 - cls_loss: 1.2035 - box_loss: 0.0026 - model_loss: 1.3320 - val_total_loss: 1.2406 - val_cls_loss: 1.1423 - val_box_loss: 7.0817e-04 - val_model_loss: 1.1777
Epoch 3/30
7/7 [==============================] - 9s 1s/step - total_loss: 1.2366 - cls_loss: 1.0967 - box_loss: 0.0015 - model_loss: 1.1737 - val_total_loss: 1.1167 - val_cls_loss: 1.0246 - val_box_loss: 5.8479e-04 - val_model_loss: 1.0538
Epoch 4/30
7/7 [==============================] - 10s 1s/step - total_loss: 0.9968 - cls_loss: 0.8808 - box_loss: 0.0011 - model_loss: 0.9339 - val_total_loss: 0.9243 - val_cls_loss: 0.8260 - val_box_loss: 7.0774e-04 - val_model_loss: 0.8614
Epoch 5/30
7/7 [==============================] -

In [ ]:
loss, coco_metrics = model.evaluate(validation_data, batch_size=4)
print(f"Validation loss: {loss}")
print(f"Validation coco metrics: {coco_metrics}")


In [ ]:
model.export_model()

In [ ]:
qat_hparams = object_detector.QATHParams(learning_rate=0.3, batch_size=4, epochs=10, decay_steps=6, decay_rate=0.96)
model.quantization_aware_training(train_data, validation_data, qat_hparams=qat_hparams)
qat_loss, qat_coco_metrics = model.evaluate(validation_data)
print(f"QAT validation loss: {qat_loss}")
print(f"QAT validation coco metrics: {qat_coco_metrics}")


In [ ]:
new_qat_hparams = object_detector.QATHParams(learning_rate=0.9, batch_size=4, epochs=15, decay_steps=5, decay_rate=0.96)
model.restore_float_ckpt()
model.quantization_aware_training(train_data, validation_data, qat_hparams=new_qat_hparams)
qat_loss, qat_coco_metrics = model.evaluate(validation_data)
print(f"QAT validation loss: {qat_loss}")
print(f"QAT validation coco metrics: {qat_coco_metrics}")


In [ ]:
model.export_model('model_int8_qat.tflite')

In [ ]:
from mediapipe_model_maker import quantization
quantization_config = quantization.QuantizationConfig.for_float16()
model.restore_float_ckpt()
model.export_model(model_name="model_fp16.tflite", quantization_config=quantization_config)